In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [2]:
import pandas as pd
import numpy as np
from geopy.distance import geodesic

In [ ]:
# Define the dataset path
dataset_path = "/content/drive/MyDrive/개발/코레할랄 맵/V1/Restaurant_1129.csv"

# Load the dataset
data = pd.read_csv(dataset_path)

# Function to find nearest locations
def find_nearest_locations(user_lat, user_lon, category, dataset, max_results=10):
    # Filter by category
    filtered = dataset[dataset['Category'] == category].copy()

    if filtered.empty:
        return []

    # Calculate distances
    filtered['Distance'] = filtered.apply(
        lambda row: geodesic((user_lat, user_lon), (row['Latitude'], row['Longitude'])).km, axis=1
    )

    filtered['Operating Hours'] = dataset["Time"]

    # Sort by distance and return top results
    sorted_results = filtered.sort_values(by='Distance').head(max_results)
    return sorted_results[['Name', 'Distance', 'Operating Hours']]


# Example
user_lat = 37.551170   # Namsan Tower
user_lon = 126.988228
category = "Arabic"

# Find top 3 locations
results = find_nearest_locations(user_lat, user_lon, category, data)

# Display results
print("Top 10 nearest locations:")
print(results)


Top 10 nearest locations:
                     Name  Distance  \
350            Chickpeace  1.204305   
49     Saffron Restaurant  1.767240   
18       Petra Restaurant  1.836158   
45           Petra Palace  1.847492   
344            Grill Doha  1.862746   
40                  Dubai  1.963789   
19   Arabesque Restaurant  1.966447   
15     Zaffran Restaurant  2.184614   
12              Cairo BBQ  2.212695   
348        Istanbul Grill  3.630738   

                                       Operating Hours  
350                                 10:30am - 08:00pm.  
49                                  11:30am - 10:00pm.  
18                                 11:30 am - 10:00 pm  
45            11:30am - 08:30pm. Closed on Wednesdays.  
344  11:30am - 09:50pm. Break time 2:30pm - 5:00pm....  
40                               12:00 pm - 11:00 pm.   
19                               11:00 am - 10:00 pm.   
15                                 12:00 pm - 10:00 pm  
12                             

In [ ]:
!pip install openai

# !pip install openai==0.28

In [ ]:
from openai import OpenAI
from google.colab import drive, userdata
import pandas as pd
from geopy.distance import geodesic

# Define the dataset path
dataset_path = "/content/drive/MyDrive/개발/코레할랄 맵/V1/Restaurant_1129.csv"

client = OpenAI(api_key=userdata.get('api_key'))

# Load the dataset
data = pd.read_csv(dataset_path)

# Function to infer category from text
def infer_category(prompt):
    prompt = prompt.lower()
    print(len(prompt))
    if "uzbek" in prompt:
        return "Uzbek"
    elif "arabic" in prompt:
        return "Arabic"
    elif "restaurant" in prompt:
        return "Restaurant"
    # Add more categories as needed
    return None

# Function to find nearest locations
def find_nearest_locations(user_lat, user_lon, category, dataset, max_results=2):
    # Filter by category
    filtered = dataset[dataset['Category'] == category].copy()

    if filtered.empty:
        return []

    # Calculate distances
    filtered['distance'] = filtered.apply(
        lambda row: geodesic((user_lat, user_lon), (row['Latitude'], row['Longitude'])).km, axis=1
    )

    # Sort by distance and return top results as a list of dictionaries
    sorted_results = filtered.sort_values(by='distance').head(max_results)
    return sorted_results[['Name', 'distance', 'Time']].to_dict(orient='records')



def generate_response_with_gpt(user_lat, user_lon, text):
    # Infer category from text
    category = infer_category(text)
    if not category:
        return "Unable to infer category from the prompt. Please provide more details."

    results = find_nearest_locations(user_lat, user_lon, category, data)

    if not results:
        return "No matching locations found."

    # Prepare a structured prompt for GPT
    places_summary = "\n".join(
        f"{index + 1}. {place['Name']} (Distance: {place['distance']:.2f} km, Hours: {place['Time']})"
        for index, place in enumerate(results)
    )
    prompt = f""" You are a helpful assistant. A user has requested suggestions for {category} locations.
                  Based on the search, here are the nearest options:

                  {places_summary}

                  Please format this into a user-friendly response.
                  """
    try:
        response = client.chat.completions.create(model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": prompt},
        ],
        max_tokens=100)
        return response.choices[0].message.content.strip()
    except Exception as e:
        return f"Error generating response: {e}"

# Example usage
user_lat = 37.551170   # Namsan Tower
user_lon = 126.988228
text = "Find nearest Arabic restaurant by distance"
response = generate_response_with_gpt(user_lat, user_lon, text)
print(response)


42
Certainly! Here are some nearby Arabic locations for you to consider:

1. Chickpeace
   - Distance: 1.20 km 
   - Operating Hours: 10:30 am - 08:00 pm 

2. Saffron Restaurant
   - Distance: 1.77 km 
   - Operating Hours: 11:30 am - 10:00 pm 

I hope you find these options helpful for your search!


In [ ]:
# Define the dataset path
dataset1 = "/content/drive/MyDrive/개발/코레할랄 맵/V1/Restaurant_1129.csv"
dataset2 = "/content/drive/MyDrive/개발/코레할랄 맵/V1/Hotel_1127.csv"

# Load the datasets
data1 = pd.read_csv(dataset1)
data2 = pd.read_csv(dataset2)

# Add the necessary columns to dataset2
data2['Classification'] = 'Hotel'
data2['Category'] = None
data2['Time'] = None

# Drop the 'WKT' column from both datasets if it exists
if 'WKT' in data1.columns:
    data1 = data1.drop(columns=['WKT'])
if 'WKT' in data2.columns:
    data2 = data2.drop(columns=['WKT'])

combined_data = pd.concat([data1, data2], ignore_index=True)
combined_data.to_csv("/content/combined_dataset.csv", index=False)

In [ ]:
dataset1 = "/content/combined_dataset.csv"
dataset2 = "/content/drive/MyDrive/개발/코레할랄 맵/V1/Pray_1127.csv"

# Load the datasets
data1 = pd.read_csv(dataset1)
data2 = pd.read_csv(dataset2)

# Add the necessary columns to dataset2
data2['Classification'] = 'Prayer'
data2['Category'] = None

if 'WKT' in data2.columns:
    data2 = data2.drop(columns=['WKT'])

combined_data = pd.concat([data1, data2], ignore_index=True)
combined_data.to_csv("/content/combined_dataset.csv", index=False)

In [ ]:
dataset = "/content/combined_dataset.csv"

# Load the datasets
data = pd.read_csv(dataset)
print(data)

       Latitude   Longitude                               Name Category  \
0     37.533220  126.996389                                Eid   Korean   
1     37.534410  126.995378          Mr.Kebab (Itaewon Branch)  Turkish   
2     37.534028  126.994217                       Sultan Kebab  Turkish   
3     37.534502  126.997047              Kervan Turkish Bakery  Turkish   
4     37.534327  126.995358                             Kervan  Turkish   
...         ...         ...                                ...      ...   
1595  36.393402  127.399507               Prayer room in KAIST      NaN   
1596  36.336314  127.458233  Prayer room in Daejeon University      NaN   
1597  37.559249  126.924394               Prayer romm in IFTAR      NaN   
1598  37.565702  127.005632      Prayer room in Star samarkand      NaN   
1599  37.555850  126.925240         Prayer room in Nusantaraku      NaN   

                                         Time  \
0      12:00 pm - 09:00 pm. closed on Tuesday   
1

In [ ]:
from openai import OpenAI
from google.colab import userdata

client = OpenAI(api_key=userdata.get('api_key2'))

def generate_response_with_gpt(prompt):
    try:
        # Generate the response in the inferred language
        instruction = f"You are a helpful assistant. Reply back in the language user is using."
        response = client.chat.completions.create(
            model="gpt-4-turbo",
            messages=[
                {"role": "system", "content": instruction},
                {"role": "user", "content": prompt}
            ],
            max_tokens=1000
        )
        return response.choices[0].message.content.strip()
    except Exception as e:
        return f"Error generating response: {e}"

prompt = "내일 인천국제공항에 도착하는데, 할랄인들을 위한 방문 가능한 장소를 추천해주세요"
response = generate_response_with_gpt(prompt)
print(response)


인천국제공항에 도착하시면 방문하실 수 있는 할랄 친화적인 장소들을 추천드릴게요:

1. **인천 중구의 한국슬람평화의사원** - 인천에 위치한 이 사원은 무슬림 관광객들이 기도를 할 수 있는 공간을 제공합니다.

2. **할랄 식당** - 인천 국제공항 인근에는 여러 할랄 인증 식당이 있습니다. 예를 들어, '우스마니아'는 인기 있는 할랄 인증 파키스탄 식당입니다.

3. **송도 센트럴파크** - 송도에 위치한 이 공원은 아름다운 경관을 자랑하며, 가족 단위 방문객들에게 적합한 장소입니다.

4. **인천 차이나타운** - 다양한 문화와 음식을 경험할 수 있는 곳으로, 일부 식당에서는 할랄 음식도 제공합니다.

5. **트리플스트리트 쇼핑몰** - 다양한 쇼핑 옵션과 식사 장소를 제공하며, 할랄 음식 옵션이 있는 식당도 찾을 수 있습니다.

이러한 장소들을 방문하시면서 할랄 요구사항에 맞는 식사와 기도 공간 등을 찾으실 수 있을 거예요. 좋은 여행 되세요!


In [ ]:
import numpy as np
import pandas as pd

# Define the dataset path
dataset1 = "/content/Restaurant_1129_Fake.csv"
dataset2 = "/content/Seoul_1127_Fake_Address.csv"

# we care about Name, Description, Address columns for now
# later additional paramters can be added such as query by classification
# later, for specific details, we either 1) fine tune, 2) ask clarification, guess intent (detail) and then query

# Load the datasets
data1 = pd.read_csv(dataset1)
data2 = pd.read_csv(dataset2)

# Add the necessary columns to dataset1
data1['Type'] = 'Restaurant'

# Add the necessary columns to dataset2
data2['Type'] = 'Tourist Attraction'
data2['Category'] = None
data2['Time'] = None

# Drop the 'WKT' column from both datasets if it exists
if 'WKT' in data1.columns:
    data1 = data1.drop(columns=['WKT'])
if 'Place' in data2.columns:
    data2 = data2.drop(columns=['Place'])

# Combine the datasets
combined_data = pd.concat([data1, data2], ignore_index=True)

# Remove rows where 'Description' is empty or null
combined_data = combined_data.dropna(subset=['Description'])

# Replace any remaining null, empty, or whitespace-only values with 'N/A'
combined_data = combined_data.replace(to_replace=[np.nan, '', ' '], value='N/A')

# Save the cleaned dataset (optional, only if needed for external use)
combined_data.to_csv("/content/combined_dataset.csv", index=False)

In [ ]:
from transformers import pipeline

# Load a zero-shot classification pipeline
classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")

# Define candidate labels
labels = ["Tourist Attraction", "Restaurant", "Prayer Room"]

# Example prompt
prompt = "I will be coming to Seoul this Friday, make a detailed 1 day itinerary, including tourist attractions and Halal Restaurants and prayer rooms"

# Classify the prompt
result = classifier(prompt, candidate_labels=labels)
print(result)


Device set to use cpu


{'sequence': 'I will be coming to Seoul this Friday, make a detailed 1 day itinerary, including tourist attractions and Halal Restaurants and prayer rooms', 'labels': ['Tourist Attraction', 'Prayer Room', 'Restaurant'], 'scores': [0.5679824948310852, 0.2610222101211548, 0.17099523544311523]}


In [ ]:
import requests

def get_coordinates_nominatim(place_name):
    base_url = "https://nominatim.openstreetmap.org/search"
    params = {
        "q": place_name,
        "format": "json",
        "limit": 1
    }
    headers = {
        "User-Agent": "ExampleMap/1.0 (example@example.com)"
    }
    response = requests.get(base_url, params=params, headers=headers)

    if response.status_code == 200:
        data = response.json()
        if data:
            return float(data[0]['lat']), float(data[0]['lon'])
        else:
            raise ValueError(f"No results found for {place_name}")
    else:
        raise ValueError(f"Failed to fetch data for {place_name}, Status Code: {response.status_code}")

# Example usage
try:
    lat, lng = get_coordinates_nominatim("Incheon International Airport")
    print("Latitude:", lat, "Longitude:", lng)
except ValueError as e:
    print(e)


Latitude: 37.4634593 Longitude: 126.44170925544927


In [ ]:
import json
import requests
from google.colab import userdata
from openai import OpenAI

client = OpenAI(api_key=userdata.get('api_key2'))

def infer_type_and_location(prompt):
    # Default coordinates for Seoul Station
    DEFAULT_LAT = 37.551170
    DEFAULT_LON = 126.988228
    DEFAULT_LOCATION = "Seoul Station, South Korea"

    # Prepare GPT prompt
    gpt_prompt = f"""
    You are an AI assistant specializing in Muslim-friendly places in Korea.
    Analyze the following user prompt: "{prompt}".
    Return a JSON object with:
    - 'type': an array of classifications ('Restaurant' and/or 'Tourist Attraction').
      Include both if both are mentioned; otherwise, use only the relevant type.
      Default to ['Restaurant', 'Tourist Attraction'] if ambiguous.
    - 'location': A specific location mentioned in the prompt, if any.
      Otherwise, default to '{DEFAULT_LOCATION}'.
    - 'coordinates': A dictionary with 'latitude' and 'longitude' values.
      For 'Seoul' or 'South Korea', always return {{'latitude': 37.551170, 'longitude': 126.988228}}.
    """

    try:
        response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[
                {"role": "system", "content": "You are a helpful assistant."},
                {"role": "user", "content": gpt_prompt},
            ],
            max_tokens=150
        )

        result = response.choices[0].message.content.strip()
        print("Debug - Output:", result)

        if result.startswith("```"):
            result = result.split("```")[1].strip()

        # Parse GPT response as JSON
        parsed_result = json.loads(result)

        # Extract classification and location
        classification = parsed_result.get("type", ["Restaurant", "Tourist Attraction"])
        location = parsed_result.get("location", DEFAULT_LOCATION)
        coordinates = parsed_result.get("coordinates", {
            "latitude": DEFAULT_LAT,
            "longitude": DEFAULT_LON
        })

        # If location is mentioned and not default, use Nominatim for geocoding
        if location != DEFAULT_LOCATION or "Seoul" not in location or "South Korea" not in location:
            try:
                lat, lon = get_coordinates_nominatim(location)
                coordinates = {"latitude": lat, "longitude": lon}
            except Exception as e:
                print(f"Geocoding failed for {location}: {e}")
                coordinates = {"latitude": DEFAULT_LAT, "longitude": DEFAULT_LON}  # Fallback

        if not location:
            location = {"latitude": DEFAULT_LAT, "longitude": DEFAULT_LON}

        return {
            "type": classification,
            "location": location,
            "coordinates": coordinates
        }

    except json.JSONDecodeError as jde:
        raise ValueError(f"Failed to parse GPT response as JSON. Raw output: {result}. Error: {str(jde)}")
    except Exception as e:
        raise ValueError(f"Failed to infer classification and location: {str(e)}")

# Example usage
prompt = "I will be arriving at Incheon International Airport tomorrow. Suggest Halal restaurants nearby."
output = infer_type_and_location(prompt)

Debug - Output: {
    "type": ["Restaurant"],
    "location": "Incheon International Airport, South Korea",
    "coordinates": {
        "latitude": 37.460190,
        "longitude": 126.440695
    }
}


In [ ]:
print(output["type"])
print(output["location"])
print(output["coordinates"])

['Restaurant', 'Tourist Attraction']
Seoul
{'latitude': 37.5666791, 'longitude': 126.9782914}


In [ ]:
import pandas as pd
from geopy.distance import geodesic
import json

def find_nearest_locations(data, user_lat, user_lon, place_type, max_results):
    """
    Finds the nearest locations for a given type and returns a list of places with Name, Address, Description, and Distance.
    """
    filtered = data[data['Type'].str.lower() == place_type.lower()]

    if filtered.empty:
        return []

    filtered = filtered.copy()
    filtered['distance_km'] = filtered.apply(
        lambda row: geodesic((user_lat, user_lon), (row['Latitude'], row['Longitude'])).km, axis=1
    )

    # Sort by distance and limit the number of results
    nearest_places = filtered.sort_values(by='distance_km').head(max_results)

    nearest_places = nearest_places[['Name', 'Address', 'Description', 'Type', 'distance_km']]

    nearest_places = nearest_places.fillna('N/A')

    return nearest_places.to_dict(orient='records')

def get_places_by_type(data, types, user_location, limit=2):
    f"""
    Gets places by type (e.g., Restaurants, Tourist Attractions).
    Returns "{limit}" nearest places for each type with distance and outputs them as a JSON file.
    """
    user_lat, user_lon = user_location
    results = []

    for place_type in types:
        nearest_places = find_nearest_locations(data, user_lat, user_lon, place_type, max_results=limit)
        results.extend(nearest_places)

    # Output results to JSON
    output_file = '/content/places_by_type.json'
    with open(output_file, 'w') as f:
        json.dump(results, f, indent=4, ensure_ascii=False)

    print(f"Results saved to {output_file}")
    return results

# Example usage
data_path = '/content/combined_dataset.csv'
data = pd.read_csv(data_path)


user_location = (37.551170, 126.988228)  # Seoul Tower coordinates
types = ['Restaurant', 'Tourist Attraction']
places = get_places_by_type(data, types, user_location, limit=2)

print(json.dumps(places, indent=4, ensure_ascii=False))

Results saved to /content/places_by_type.json
[
    {
        "Name": "By Tofu",
        "Address": "South Korea, Seoul, Yongsan District, yongsan-dong 2(i)-ga, 1-467번지 1층",
        "Description": "Vegetarian Restaurant.",
        "Type": "Restaurant",
        "distance_km": 0.6315577473665679
    },
    {
        "Name": "Nuldam Space",
        "Address": "South Korea, Seoul, 용산동2가 19-12번지 1층 용산구 서울특별시 KR",
        "Description": "Vegetarian Cafe.",
        "Type": "Restaurant",
        "distance_km": 0.7752159035053967
    },
    {
        "Name": "Namsan Seoul Tower (남산서울타워)",
        "Address": "644 Seolleung-ro, Jung-gu, Changwon, South Korea, 48368",
        "Description": "Admission Fees:[ Observatory] Adults 16,000 won / Children 12,000 won\n<br><br>\n[Namsan Cable Car]<br>\n- Operating hours: 10:00-23:00<br>\n- Round-trip fare: Adults 14,000 won / Children 10,500 won<br>\n- One way: Adults 11,000 won / Children 8,000 won \nRestrooms:Available\nAvailable Facilities:[Seoul Tower

In [ ]:
def generate_human_response(json_file_path):
    """
    Reads the JSON file and generates a human-readable response using GPT.
    """
    with open(json_file_path, 'r') as f:
        places = json.load(f)

    gpt_prompt = "You are a travel assistant. Always include the full address in your responses. Divide the plan for lunch and dinner where each has one tourist attraction and restaurant. If the distance is less than 1 km, then return distnace in meters instead. Describe the following places for a user planning their day:\n\n"

    for place in places:
        name = place['Name']
        address = place['Address']
        description = place['Description']
        distance = round(place['distance_km'], 1)
        place_type = place['Type']

        gpt_prompt += (
            f"- {place_type}: {name}\n"
            f"  Address: {address}\n"
            f"  Distance: {distance} km from your location.\n"
            f"  Description: {description}\n\n"
        )

    try:
        response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[
                {"role": "system", "content": "You are a helpful travel assistant."},
                {"role": "user", "content": gpt_prompt},
            ],
            max_tokens=1000,
            # temperature=0.7
        )

        return response.choices[0].message.content.strip()

    except Exception as e:
        raise ValueError(f"Failed to generate human response: {str(e)}")

# Example usage
places_json_path = '/content/places_by_type.json'
response = generate_human_response(places_json_path)
print(response)


For your day in Seoul, here is a plan for lunch and dinner with a tourist attraction included for each:

Lunch:
Restaurant: **By Tofu**
Address: 1-467번지 1층, Yongsan-dong 2(i)-ga, Yongsan District, Seoul, South Korea
Distance: 600 meters from your location
Description: Vegetarian Restaurant

Tourist Attraction: **Namsan Seoul Tower (남산서울타워)**
Address: 644 Seolleung-ro, Jung-gu, Changwon, Seoul, South Korea, 48368
Description: Namsan Seoul Tower offers breathtaking views of the city. Explore the observatory and facilities available on different floors. Don't miss the Namsan Cable Car experience.

Dinner:
Restaurant: **Nuldam Space**
Address: 19-12번지 1층, Yongshan Dong 2-ga, Seoul, South Korea
Distance: 800 meters from your location
Description: Vegetarian Cafe

Tourist Attraction: **Namsan Seoul Tower Hanbok Culture Experience Center (남산서울타워 한복문화체험관)**
Address: 480 Jongno, Seodaemun-gu, Goyang, South Korea, 30333
Description: Immerse yourself in Korean culture at the Hanbok Culture Experi

In [ ]:
session_history = [
    {"role": "system", "content": "You are a helpful travel assistant."}
]

def add_to_session_history(role, content):
    session_history.append({"role": role, "content": content})

def generate_response_with_session(prompt):
    add_to_session_history("user", prompt)

    try:
        response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=session_history,
            max_tokens=100,
            temperature=0.7
        )

        gpt_response = response.choices[0].message.content.strip()

        # Add response to session history
        add_to_session_history("assistant", gpt_response)

        return gpt_response

    except Exception as e:
        raise ValueError(f"Failed to generate response: {str(e)}")


In [ ]:
initial_prompt = "Please remember my name is John Middle"
print(generate_response_with_session(initial_prompt))

Got it, John Middle! If you have any more questions or need further assistance with your travel plans, feel free to ask.


In [ ]:
clarification_prompt = "What is my name again? I had amnesia recently"
print(generate_response_with_session(clarification_prompt))


Your name is John Middle. If you need any help or information, feel free to ask, John!


In [3]:
!pip install rapidfuzz

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 27.7 MB/s eta 0:00:00


In [ ]:
from rapidfuzz import fuzz
fuzz.ratio("Eid", "Eit")

66.66666666666667

In [ ]:
from rapidfuzz import process, fuzz
import pandas as pd

data = pd.read_csv("combined_dataset.csv")

def find_closest_match(place_name, data, limit=3):
    # Use fuzzy matching to find the closest matches by name
    choices = data['Address'].tolist()
    matches = process.extract(place_name, choices, limit=limit, scorer=fuzz.WRatio)

    filtered_matches = [match for match in matches if match[1] > 50]  # Threshold = 70%
    return filtered_matches

# Example usage
place_name = "Itaewon street"
closest_matches = find_closest_match(place_name, data)
print(closest_matches)


[('176-1 Itaewon-ro, Itaewon-dong, Yongsan District, Seoul, South Korea', 57.857142857142854, 58), ('352 Itaewon-ro, Seocho-gu, Gwangju, South Korea, 75409', 57.857142857142854, 897), ('733 Itaewon-ro, Seocho-gu, Daegu, South Korea, 77373', 57.857142857142854, 1012)]
